In [1]:
# This is the main notebook to run.
# In this notebook, we go through the "attack-estimation" portion of the project.
# Some utility code, including base classes, lives in files outside this notebook.

In [2]:
# A bunch of imports
from bayes_net import BayesNet

In [3]:
# A demonstration for developers of how to create a BayesNet
my_net = BayesNet()

Created a bayes net
